# Vehicle Detection and Tracking
- by NK Zou
  
### Table of content
1- Classifier with HLS & HOG, SVC & DecisionTree  
2- Sliding Window Search  
3- Video Implementation  

1- Classifier 分类器
2- Proportional Steps 比例步骤
3- Window Tiling 窗口平铺
4- Heat Maps 热图
5- Processing Still Images 处理静态图像
6- Heat Map Combinations 热图组合
7- Vehicle Tracking 车辆跟踪
8- Bounding Box 边界框
9- Vehicle Detector 探测车辆

分类器
Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector.
Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.

窗口平铺
Implement a sliding-window technique and use your trained classifier to search for vehicles in images.

热图，探测车辆
Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
Estimate a bounding box for vehicles detected.

### 1- Classifier with HLS & HOG, SVC & DecisionTree

In [ ]:
import glob2
import numpy as np
import cv2
import pickle
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

'''
1- All of self.attribute can be shared under same class, add self. as prefix when use prameter in function
   class的作用在于在同一个clas内的所有self.attribute可以共享，parameters则在引用时添加前缀self.即可
2- If subfolder has many folders in the path, use ** instead of all folder names, like * instead of all file names
   如果路径中的包含多个子文件夹，用 **  代替所有文件夹名称， 正如用 * 代替所有文件名一样
3- __MACOSX and .DS_Store are useless files for windows users, can remove them and set path as 'folder/**/*.png'
   __MACOSX 和 .DS_Store 都是Mac压缩时为方便预览而生成的垃圾文件，windows用户可以直接删除，把路径改为 'folder/**/*.png'
4- In decisiontreeclassifier, the feature in the higher level node has a greater contribution to the final prediction
   一个决策树，节点在越高的分支，相应的特征对最终预测结果的贡献越大。这里的大，是指影响输入数据集的比例比较大
5- feature_importances_ is an attribute of sklearn, _ is a connector between words, but last _ is not implication in here
   feature_importances_ 只是sklearn的一个属性，下划线代表两个单词间的连接符，但最后一个下划线在这里没有特殊意义，只是一个习惯
'''

class Classifier:

### Paramters #################################################################

### combined_spatial()
    SPATIAL_SIZE = (32, 32)
    HLS_BINS = 512 
    HLS_BIN_RANGE = (0,255)
    HOG_ORIENTATIONS = 9
    HOG_PIXELS_PER_CELL = (8,8)
    HOG_CELLS_PER_BLOCK = (2,2)
### 用于load()
    IMAGE_SIZE = (64, 64)
    TEST_FRACTION = 0.2
    RANDOM_STATE = 12345
### train()
    SVC_KERNEL = 'rbf'
    SVC_C_VALUE = 2.0
    DT_MIN_SAMPLES_SPLIT = 40
### store(), define path 'classified_data.p' to save classified data
    PICKLE_FILE = 'classified_data.p'

### Function Set ##############################################################

### Initialization, vehicle=true, non-vehicle=false
    def __init__(self, true_path, false_path):
        self.true_path = true_path
        self.false_path = false_path

### Combine spatial function for load data
    def combined_spatial(self, image):
    ### compute spatial of image
        spatial = cv2.resize(image, self.SPATIAL_SIZE).ravel()
    ### compute spatial with hls
        hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        h_hist = np.histogram(hls[:,:,0], bins=self.HLS_BINS, range=self.HLS_BIN_RANGE)
        s_hist = np.histogram(hls[:,:,2], bins=self.HLS_BINS, range=self.HLS_BIN_RANGE)
        hls = np.concatenate((h_hist[0], s_hist[0]))
    ### compute spatial with hog
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        hogs = hog(gray, orientations=self.HOG_ORIENTATIONS, pixels_per_cell=self.HOG_PIXELS_PER_CELL, cells_per_block=self.HOG_CELLS_PER_BLOCK, transform_sqrt=True, visualise=False, feature_vector=True)
        combined_spatial = np.concatenate((spatial, hls, hogs))
        return combined_spatial

### Load and split data
    def load(self):
    ### Define the path of vehicle folder and non-vehicle folder
        true_paths = glob2.glob(self.true_path)
        false_paths = glob2.glob(self.false_path)
    ### Define features set and labels set for load images and store data
        features = []
        labels = []
    ### Load vehicle images and store features and labels, vehicle=1.0
        for filename in true_paths:
            image = cv2.resize(mpimg.imread(filename), self.IMAGE_SIZE)
            hls_hog = self.combined_spatial(image)
            features.append(hls_hog)
            labels.append(1.0)
    ### Load non-vehicle images and store features and labels, non-vehicle=0.0
        for filename in false_paths:
            image = cv2.resize(mpimg.imread(filename), self.IMAGE_SIZE)
            hls_hog = self.combined_spatial(image)
            features.append(hls_hog)
            labels.append(0.0)
    ### Normalize features. self.scaler is a glbal attribute in this class because their have prefix (self.)
        features=np.array(features, np.float32)
        self.scaler = StandardScaler().fit(features)
        features = self.scaler.transform(features)
    ### Split to train features, train labels, test features, test labels, store train labels and test labels as numpy array
        self.train_features, self.test_features, self.train_labels, self.test_labels = train_test_split(features, labels, test_size=self.TEST_FRACTION, random_state=self.RANDOM_STATE)
        self.train_labels = np.array(self.train_labels, np.float32)
        self.test_labels = np.array(self.test_labels, np.float32)
        return self

### Train data with SVC and Decision Tree
### self.svc_classifier and self.dt_classifier are glbal attribute in this class because their have prefix (self.)
### Both of self.svc_classifier and dt_classifier need to use .fit() before testing
    def train(self):
        self.svc_classifier = SVC(kernel=self.SVC_KERNEL,C=self.SVC_C_VALUE)
        self.svc_classifier.fit(self.train_features, self.train_labels)
        self.dt_classifier = DecisionTreeClassifier(min_samples_split=self.DT_MIN_SAMPLES_SPLIT)
        self.dt_classifier.fit(self.train_features, self.train_labels)
        return self

### Combine prediction function for testing, vehicle=1.0, non-vehicle=0.0
    def combined_prediction(self, list_svc, list_dt):
        prediction = []
        for index, val_svc in enumerate(list_svc):
            val_dt = list_dt[index]
            if val_svc == 1.0 or val_dt == 1.0:
                prediction.append(1.0)
            else:
                prediction.append(0.0)
        return prediction

### Test trained data set, compute accuracy
    def test(self):
        svc_prediction = self.svc_classifier.predict(self.test_features)
        svc_accuracy = accuracy_score(svc_prediction, self.test_labels)
        dt_prediction = self.dt_classifier.predict(self.test_features)
        dt_accuracy = accuracy_score(dt_prediction, self.test_labels)
        composited_prediction = self.combined_prediction(svc_prediction, dt_prediction)
        composited_accuracy = accuracy_score(composited_prediction, self.test_labels)
        return np.array([svc_accuracy, dt_accuracy, composited_accuracy])

### Store classified data
    def store(self):
        pickle.dump(self, open(self.PICKLE_FILE, 'wb'))
        
### Object identification, just like combine fuction of load and test
    def identificator(self, image):
        image = cv2.resize(image, self.IMAGE_SIZE)
        hls_hog = self.combined_spatial(image)
        features=np.array(hls_hog, np.float32)
        features = self.scaler.transform(features)
        svc_prediction = self.svc_classifier.predict(features)
        dt_prediction = self.dt_classifier.predict(features)
        composite_prediction = self.combined_prediction(svc_prediction, dt_prediction)
        return np.array([svc_prediction, dt_prediction, composite_prediction])

### Classify data #############################################################
classifier = Classifier('vehicles/**/*.png', 'non-vehicles/**/*.png')        
print(classifier.load().train().test())
classifier.store()

### 2- Sliding Window Search

In [ ]:
# 2.1- Describe how (and identify where in your code) you implemented a sliding window search. How did you decide what scales to search and how much to overlap windows?
  
# A sliding window approach has been implemented, where overlapping tiles in each test image are classified as vehicle or non-vehicle. Some justification has been given for the particular implementation chosen.

In [ ]:
# 2.2- Show some examples of test images to demonstrate how your pipeline is working. How did you optimize the performance of your classifier?
  
# Some discussion is given around how you improved the reliability of the classifier i.e., fewer false positives and more reliable car detections (this could be things like choice of feature vector, thresholding the decision function, hard negative mining etc.)

### 3- Video Implementation

In [ ]:
# 3.1- Provide a link to your final video output. Your pipeline should perform reasonably well on the entire project video (somewhat wobbly or unstable bounding boxes are ok as long as you are identifying the vehicles most of the time with minimal false positives.)
  
# The sliding-window search plus classifier has been used to search for and identify vehicles in the videos provided. Video output has been generated with detected vehicle positions drawn (bounding boxes, circles, cubes, etc.) on each frame of video.

In [ ]:
# 3.2- Describe how (and identify where in your code) you implemented some kind of filter for false positives and some method for combining overlapping bounding boxes.
  
# A method, such as requiring that a detection be found at or near the same position in several subsequent frames, (could be a heat map showing the location of repeat detections) is implemented as a means of rejecting false positives, and this demonstrably reduces the number of false positives. Same or similar method used to draw bounding boxes (or circles, cubes, etc.) around high-confidence detections where multiple overlapping detections occur.